In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

base_dir_1 = "/Data/Train"

# Tăng cường dữ liệu.
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
])

# chia dữ liệu cho train
train_ds = keras.utils.image_dataset_from_directory(
    base_dir_1,
    image_size=(200,200),
    batch_size=128,
    seed=100,
    subset='training',
    validation_split=0.2
)

# Lưu tên lớp trước trước khi chuyển đổi map
class_names = train_ds.class_names
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

# chuẩn bị dữ liệu cho tập kiểm định chéo
val_ds = keras.utils.image_dataset_from_directory(
    base_dir_1,  # Sử dụng thư mục riêng cho tập kiểm định
    seed=1,
    image_size=(200,200),
    batch_size=128,
    subset='validation',
    validation_split=0.06
)

# chuẩn bị dữ liệu tập test model
test_ds = keras.utils.image_dataset_from_directory(
    base_dir_1,
    seed=2,
    image_size=(200,200),
    batch_size=128,
    subset='validation',
    validation_split=0.14
)

#số lớp trong dữ liệu.
num_classes = len(class_names)

# Xây dựng mô hình
model = keras.models.Sequential([
    keras.layers.Rescaling(1./255, input_shape=(200,200,3)),
    keras.layers.Conv2D(filters=16, kernel_size=(3,3),padding='valid',activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=32, kernel_size=(4,4),padding='same',activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(4,4)),
    keras.layers.Conv2D(filters=32, kernel_size=(5,5),padding='same',activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(4,4)),
    keras.layers.Conv2D(filters=64, kernel_size=(5,5),padding='same',activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(3,3)),
    keras.layers.Flatten(),
    keras.layers.Dropout(rate=0.4),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(units=64,activation='relu'),
    keras.layers.Dense(units=256,activation='relu'),
    keras.layers.Dense(units=num_classes, activation='softmax')
])

model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

model.fit(train_ds,epochs=55,validation_data=val_ds)
print("Training finished")


**Đánh giá tôngr thể mô hình sau khi train qua tập dữ liệu kiểm tra**

In [ ]:
#Đánh giá mô hình trên tập dữ liệu kiểm tra
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc}")

**Lưu model sau khi khi huấn luyện**

In [ ]:
# Lưu model
model.save("/Train/model-flower14")